# Modello ANNNI - Evoluzione temporale e correlazioni
Luca Driussi, Carlo Tortora, Beatrice Volpato
<br>
Progetto di Computazione Quantistica
<br>
Anno 2024/25

## Introduzione

Il seguente progetto d'esame è dedicato allo studio del modello ANNNI (Axial-Next-Nearest-Neighbor-Ising). Questo modello, particolarmente interessante per l'analisi di transizioni di fase e fenomeni di frustrazione magnetica, è un'estensione del modello di Ising  che include interazioni tra spin secondi vicini.
Studiamo il modello ANNNI per una catena unidimensionale di L spin, per consideriamo un'Hamiltoniana della forma:

$$
H_{ANNNI}=-J\sum_{i}(\sigma_z^i\sigma_z^{i+1}-\kappa\sigma_z^i\sigma_z^{i+2}+h\sigma_x^i).
$$

Il sistema è unidimensionale, dove $\sigma^i_\alpha$ sono le matrici di Pauli lungo la direzione i-esima dello spin,  $\sigma^i_\alpha\in \{x,y,z\} $ e assumiamo gli estremi della catena liberi. La scala di energia dell'Hamiltoniana è determinata dalla costante di accoppiamento 
J (senza perdere generalità poniamo $J=1$), mentre i parametri adimensionali $\kappa$ e $h$ rappresentano rispettivamente l'interazione con il secondo vicino e con il campo magnetico trasverso. Ci limitiamo ai casi con $\kappa\geq 0$, $h\geq 0$.
Facendo uso del framework Qiskit, l'obiettivo principale di questo lavoro è stato analizzare l'evoluzione temporale di alcune osservabili di interesse con due approcci:

- Evoluzione esatta: ottenuta diagonalizzando l'Hamiltoniana del sistema;

- Trotterizzazione all’ordine 2: permette di approssimare l'evoluzione temporale sfruttando la decomposizione di Suzuki-Trotter.

Oltre l'energia totale del sistema, le osservabili studiate sono la magnetizzazione per spin lungo $z$, lungo $x$ e la correlazione tra spin $i$-esimo e spin $j$-esimo:

$$
M_z=\frac{1}{L}\sum_i \sigma_z^i
$$
$$
M_x=\frac{1}{L}\sum_i \sigma_x^i
$$
$$
C(i,j)=<\sigma_x^i\sigma_x^{i+j}>-<\sigma_x^i><\sigma_x^{i+j}>.
$$

## Diagramma di fase

Il diagramma di fase a $T = 0K$ presenta tre fasi separate da due transizioni di fase del secondo ordine:

- Fase ferromagnetica ($k<0.5$): l'interazione tra primi vicini è quella dominante e tutti gli spin sono allineati nella direzione $z$, questo implica una magnetizzazione $M_z$ uniforme. 
- Fase paramagnetica: il campo magnetico trasverso domina, causando l'allineamento degli spin nella direzione $x$. Dunque la magnetizzazione lungo $x$ si rivela un parametro d'ordine rilevante per l'identificazione di transizioni di fase. 
- Fase antiferromagnetica ($k>0.5$)



## Evoluzione temporale

### Trotterizzazione

L'evoluzione del sistema viene simulata con il metodo della trotterizzazione al secondo ordine. In breve, questa tecnica consiste nell'applicazione ripetuta di un certo gate quantistico; questa operazione approssima l'evoluzione temporale del sistema nell'intervallo di tempo discreto $dt$. A tal fine si utilizza un circuito ideale, ovvero privo di rumore. L'evoluzione trotterizzata è basata sul fatto di poter esprimere l'Hamiltoniana del sistema come una somma di termini $H_j$ costituiti da operatori di Pauli. La particolare formula utilizzata dipenderà dall'ordine di trotterizzazione scelto, se consideriamo i primi due ordini: 

- Formula di Lie-Trotter al primo ordine:
$$
e^{-iHt} = U_1 + \mathcal{O}(dt^2) = \left(\prod_{j=1}^M e^{-i a_j H_j dt} \right)^{steps} + \mathcal{O}(dt^2) 
$$
- Formula di Suzuki-Trotter al secondo ordine:
$$
e^{-iHt} = U_2 + \mathcal{O}(dt^3) = \left(\prod_{j=1}^M e^{-i a_j H_j dt/2} \prod_{j=M}^1 e^{-i a_j H_j dt/2} \right)^{steps} + \mathcal{O}(dt^3) 
$$

dove abbiamo il numero di steps di trotterizzazione definito come $steps = t/dt$, in cui $dt$ è la lunghezza del singolo intervallo.


## Analisi degli errori
Una quantità interessante da studiare è l'errore introdotto dalla trotterizzazione, il quale è definito come
$$
||U-e^{-iHt}||\leq \epsilon
$$
Dove $e^{-iHt}$ è l'operatore di evoluzione esatto e $U$ è l'operatore che implementa l'evoluzione trotterizzata, a un generico ordine.
Per un'Hamiltoniana della forma $H=A+B$ l'errore introdotto è quantificato dalle seguenti formule:

- Al primo ordine di trotterizzazione:    
 $$|
 |U_1-e^{-iHt}||\leq \frac{t^2}{2}||[B,A]||
 $$
- Al secondo ordine di trotterizzazione:        
$$
||U_2-e^{-iHt}||\leq \frac{t^3}{12}||[B,[B,A]]||+\frac{t^3}{24}||[A,[A,B]]||
$$

Nel nostro caso, concentrandoci sul secondo ordine e prendendo 

$A=-J\sum_{i}(\sigma_z^i\sigma_z^{i+1}-\kappa\sigma_z^i\sigma_z^{i+2})$ 

$B=-J\sum_{i}h\sigma_x^i$

otteniamo che l'errore scala cubicamente nel tempo di evoluzione t e quadraticamente nell'intensità del campo magnetico esterno h:
$$
||U_2-e^{-iHt}|| \lesssim t^{3}h^{2}
$$

## Implementazione del modello

In questa sezione descriviamo l'implementazione del modello ANNNI utilizzando il framework Qiskit.
Per prima cosa, viene costruita l'Hamiltoniana del sistema e vengono definite le osservabili di interesse. Dopo aver importato le librerie necessarie, tra cui $\texttt{NumPy}$, $\texttt{Qiskit}$, $\texttt{Matplotlib}$ e $\texttt{SciPy}$, vengono definiti i parametri del sistema fisico. In particolare, viene definito il numero di spin L, a forza dell'interazione tra primi vicini J, l'intensità del campo magnetico trasverso h e il coefficiente di interazione tra secondi vicini k. Questi parametri determinano il comportamento della catena di spin e influenzano la sua evoluzione temporale.

In [ ]:
import numpy as np
import qiskit
from qiskit import QuantumCircuit
from qiskit.circuit.library import PauliEvolutionGate
from qiskit.synthesis import SuzukiTrotter
from qiskit.quantum_info import SparsePauliOp
from qiskit.quantum_info import Statevector
from qiskit.primitives import Estimator
from qiskit.primitives import StatevectorEstimator
from qiskit_algorithms.time_evolvers import TrotterQRTE, TimeEvolutionProblem
import matplotlib.pyplot as plt
import scipy as sc
from scipy import linalg


# --- Parametri del sistema ---

L = 6                                                               # Numero di spin
J = 1                                                               # Forza delle interazioni
h = 0.5                                                             # Interazione con il campo esterno
k = 0.5                                                             # Interazione tra secondi vicini


# --- Costruzione di H con SparsePauliOp ---

ZZ1_list = [("ZZ", [i, i + 1], -J) for i in range(L - 1)]
ZZ2_list = [("ZZ", [i, i + 2], J*k) for i in range(L - 2)]
X_list   = [("X", [i], -J*h) for i in range(L)]
H = SparsePauliOp.from_sparse_list([*ZZ1_list, *ZZ2_list, *X_list], num_qubits=L).simplify()


# --- Costruzione delle osservabili ---

magnetization_z_op = SparsePauliOp.from_sparse_list(                # Magnetizzazione lungo z
    [("Z", [i], 1.0) for i in range(0, L)], num_qubits=L
)


magnetization_x_op = SparsePauliOp.from_sparse_list(                # Magnetizzazione lungo x
    [("X", [i], 1.0) for i in range(0, L)], num_qubits=L
)

## Studio delle correlazioni